In [1]:
notebookName = 'audio-deepfake-detection-testing'
# runJobId = 'ASVspoof-2019_small-eval-1'
# runJobId = 'ASVspoof-2019_small-eval-2'
runJobId = 'ASVspoof-2019_eval-1'
# runJobId = 'ASVspoof-2019_training'     # This should fail

import joblib
import numpy as np
from tensorflow.keras.utils import to_categorical

import configuration.configuration as configuration
from mel_spectrogram.mel_spectrogram import MelSpectrogramGenerator
from notebook_utils import notebookToPython
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor
from readers.label_reader import readTrainingLabelsWithJob

In [2]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

import json
prettyJson = json.dumps(job.__dict__, indent=4)
print(f"job: {prettyJson}")

if (job.newModelGenerated):
    raise ValueError("This notebook is meant for testing. Select a job with a value for 'persisted-model' set.")

Write python file
Using configured model name: ASVspoof-2019-1_2025-03-19T22-03-42.178503.libjob
Assigned model name: ASVspoof-2019-1_2025-03-19T22-03-42.178503.libjob
job: {
    "jobId": "ASVspoof-2019_eval-1",
    "inputFileBatchSize": 1000,
    "dataPathRootRaw": "$HOMEDRIVE$HOMEPATH/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathRoot": "C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathSuffix": "LA/ASVspoof2019_LA_eval/flac",
    "dataExtension": ".flac",
    "labelFilename": "LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt",
    "executeToCategoricalForLabels": true,
    "numClasses": 2,
    "sampleRate": 16000,
    "duration": 5,
    "numMels": 128,
    "maxTimeSteps": 109,
    "optimizer": "adam",
    "loss": "categorical_crossentropy",
    "metrics": [
        "accuracy"
    ],
    "batchSize": 32,
    "numEpochs": 10,
    "newModelGenerated": false,
    "persistedModel": "ASVspoof-2019-1_2025-03-19T22-03-42.178503.libjob",
    "persisted

In [3]:
generator = MelSpectrogramGenerator()
model = joblib.load(job.persistedModel)
evaluationProc = BasicModelEvaluationProcessor(job, model)

In [4]:
fullDataPath = job.fullJoinFilePath(job.dataPathRoot, job.dataPathSuffix)
labels = readTrainingLabelsWithJob(job)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt...


In [5]:
def processArrays(X, y):
    _X = np.array(X)
    _y = np.array(y)
    evaluationProc.process(_X, _y)


In [6]:
X = []
y = []

for filename, label in labels.items():
    _X, _y = generator.generateMelSpectrogram(job, fullDataPath, filename, label)
    X.append(_X)
    y.append(_y)

    if (len(X) >= job.inputFileBatchSize):
        processArrays(X, y)
        X = []
        y = []

if (len(X) > 0):
    processArrays(X, y)

print("\n")
evaluationProc.reportSnapshot()

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
  Batches: 1 - Files: 1000 - Score: 0.94 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
  Batches: 2 - Files: 2000 - Score: 0.942 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
  Batches: 3 - Files: 3000 - Score: 0.939 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
  Batches: 4 - Files: 4000 - Score: 0.955 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
  Batches: 5 - Files: 5000 - Score: 0.95 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
  Batches: 6 - Files: 6000 - Score: 0.95 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
  Batches: 7 - Files: 7000 - Score: 0.946 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
  Batches: 8 - Files: 8000 - Score: 0.949 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
  Batches: 9 - Files: 9000 - Score: 0.956 - Elements: 1000
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
  Batches: 10 - Files: 10000 - Score: 0.946 - Elements: 1000
32/32 ━━━━━